<a href="https://colab.research.google.com/github/LINA-LY/DetectionandTracking/blob/CODE/DetectionandTracking2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Object Detection and Tracking System for Google Colab
# Complete system with YOLO, tracking, and real-time capabilities

# ========================================
# INSTALL REQUIRED LIBRARIES
# ========================================

# Install essential packages
!pip install ultralytics  # YOLOv8
!pip install opencv-python
!pip install pillow
!pip install numpy
!pip install matplotlib
!pip install pandas
!pip install torch torchvision
!pip install supervision  # For tracking and visualization

# For video processing
!apt update &> /dev/null
!apt install ffmpeg &> /dev/null

import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import torch
from ultralytics import YOLO
import supervision as sv
from collections import defaultdict
import time
import os
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings('ignore')

# ========================================
# YOLO OBJECT DETECTION CLASS
# ========================================

class ObjectDetector:
    def __init__(self, model_size='n'):
        """
        Initialize YOLO object detector
        model_size options: 'n' (nano), 's' (small), 'm' (medium), 'l' (large), 'x' (extra large)
        """
        print(f"Loading YOLOv8{model_size} model...")
        self.model = YOLO(f'yolov8{model_size}.pt')
        self.class_names = self.model.names
        print(f"Model loaded! Can detect {len(self.class_names)} different objects.")

    def detect_objects(self, image_path_or_array, confidence=0.5):
        """Detect objects in a single image"""
        try:
            # Run detection
            results = self.model(image_path_or_array, conf=confidence)

            detections = []
            for result in results:
                boxes = result.boxes
                if boxes is not None:
                    for box in boxes:
                        # Extract box coordinates
                        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                        confidence = box.conf[0].cpu().numpy()
                        class_id = int(box.cls[0].cpu().numpy())
                        class_name = self.class_names[class_id]

                        detections.append({
                            'bbox': [int(x1), int(y1), int(x2), int(y2)],
                            'confidence': float(confidence),
                            'class_id': class_id,
                            'class_name': class_name
                        })

            return detections, results[0].plot()

        except Exception as e:
            print(f"Detection failed: {str(e)}")
            return [], None

    def show_detections(self, image_path, confidence=0.5):
        """Display image with detection results"""
        detections, annotated_image = self.detect_objects(image_path, confidence)

        if annotated_image is not None:
            plt.figure(figsize=(12, 8))
            plt.imshow(cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB))
            plt.axis('off')
            plt.title(f'Detected {len(detections)} objects')
            plt.show()

            # Print detection details
            print(f"\nDetection Results:")
            print("-" * 50)
            for i, det in enumerate(detections):
                print(f"{i+1}. {det['class_name']}: {det['confidence']:.2f}")

        return detections

# ========================================
# OBJECT TRACKING CLASS
# ========================================

class ObjectTracker:
    def __init__(self, model_size='n'):
        """Initialize object tracker with YOLO + ByteTracker"""
        self.detector = ObjectDetector(model_size)
        self.tracker = sv.ByteTrack()
        self.trace_annotator = sv.TraceAnnotator()
        self.box_annotator = sv.BoxAnnotator()
        self.label_annotator = sv.LabelAnnotator()

        # Tracking statistics
        self.track_history = defaultdict(list)
        self.object_counts = defaultdict(int)

    def process_frame(self, frame, confidence=0.5):
        """Process a single frame for detection and tracking"""
        # Run YOLO detection
        results = self.detector.model(frame, conf=confidence)[0]

        # Convert to supervision format
        detections = sv.Detections.from_ultralytics(results)

        # Update tracker
        detections = self.tracker.update_with_detections(detections)

        # Store tracking history
        for tracker_id, bbox in zip(detections.tracker_id, detections.xyxy):
            self.track_history[tracker_id].append(bbox)

        # Create labels for annotation
        labels = []
        for class_id, tracker_id, conf in zip(detections.class_id, detections.tracker_id, detections.confidence):
            class_name = self.detector.class_names[class_id]
            labels.append(f"#{tracker_id} {class_name} {conf:.2f}")

        # Annotate frame
        annotated_frame = self.box_annotator.annotate(frame.copy(), detections)
        annotated_frame = self.label_annotator.annotate(annotated_frame, detections, labels)
        annotated_frame = self.trace_annotator.annotate(annotated_frame, detections)

        return annotated_frame, detections

    def get_tracking_stats(self):
        """Get tracking statistics"""
        stats = {
            'total_tracked_objects': len(self.track_history),
            'active_tracks': len([k for k, v in self.track_history.items() if len(v) > 0]),
            'track_lengths': {k: len(v) for k, v in self.track_history.items()},
        }
        return stats

# ========================================
# VIDEO PROCESSING CLASS
# ========================================

class VideoProcessor:
    def __init__(self, model_size='n'):
        """Initialize video processor"""
        self.tracker = ObjectTracker(model_size)
        self.fps_counter = 0
        self.start_time = time.time()

    def process_video_file(self, video_path, output_path=None, max_frames=None):
        """Process video file and save results"""
        cap = cv2.VideoCapture(video_path)

        # Get video properties
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        print(f"Video info: {width}x{height}, {fps} FPS, {total_frames} frames")

        # Setup video writer if output path provided
        if output_path:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        frame_count = 0
        processed_frames = []

        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                # Process frame
                annotated_frame, detections = self.tracker.process_frame(frame)
                processed_frames.append(annotated_frame)

                # Write frame if output specified
                if output_path:
                    out.write(annotated_frame)

                frame_count += 1

                # Progress update
                if frame_count % 30 == 0:  # Every 30 frames
                    progress = (frame_count / total_frames) * 100
                    print(f"Processed {frame_count}/{total_frames} frames ({progress:.1f}%)")

                # Stop if max frames reached
                if max_frames and frame_count >= max_frames:
                    break

        finally:
            cap.release()
            if output_path:
                out.release()

        print(f"Video processing complete! Processed {frame_count} frames")

        # Return statistics
        stats = self.tracker.get_tracking_stats()
        stats['processed_frames'] = frame_count
        stats['processing_fps'] = frame_count / (time.time() - self.start_time)

        return processed_frames, stats

    def process_webcam(self, duration_seconds=30):
        """Process webcam feed (for local environments)"""
        print("Note: Webcam processing works best in local Python environments")
        print("In Colab, upload a video file instead")

        # This would work in local environment
        """
        cap = cv2.VideoCapture(0)
        start_time = time.time()

        while time.time() - start_time < duration_seconds:
            ret, frame = cap.read()
            if not ret:
                break

            annotated_frame, detections = self.tracker.process_frame(frame)

            cv2.imshow('Object Tracking', annotated_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
        """

# ========================================
# ANALYSIS AND VISUALIZATION TOOLS
# ========================================

class DetectionAnalyzer:
    def __init__(self):
        """Initialize detection analyzer"""
        self.detection_history = []

    def analyze_detections(self, detections_list):
        """Analyze detection results across multiple frames"""
        if not detections_list:
            return {}

        # Count objects by class
        class_counts = defaultdict(int)
        confidence_scores = defaultdict(list)

        for detections in detections_list:
            if hasattr(detections, 'class_id'):
                for class_id, conf in zip(detections.class_id, detections.confidence):
                    class_name = YOLO('yolov8n.pt').names[class_id]
                    class_counts[class_name] += 1
                    confidence_scores[class_name].append(conf)

        # Calculate statistics
        stats = {}
        for class_name, count in class_counts.items():
            avg_confidence = np.mean(confidence_scores[class_name])
            stats[class_name] = {
                'total_detections': count,
                'avg_confidence': avg_confidence,
                'min_confidence': min(confidence_scores[class_name]),
                'max_confidence': max(confidence_scores[class_name])
            }

        return stats

    def create_detection_report(self, stats, save_path=None):
        """Create detailed detection report"""
        if not stats:
            print("No detection statistics available")
            return

        # Create DataFrame
        df_data = []
        for class_name, class_stats in stats.items():
            df_data.append({
                'Object Class': class_name,
                'Total Detections': class_stats['total_detections'],
                'Average Confidence': f"{class_stats['avg_confidence']:.3f}",
                'Min Confidence': f"{class_stats['min_confidence']:.3f}",
                'Max Confidence': f"{class_stats['max_confidence']:.3f}"
            })

        df = pd.DataFrame(df_data)
        df = df.sort_values('Total Detections', ascending=False)

        print("📊 OBJECT DETECTION ANALYSIS REPORT")
        print("=" * 50)
        print(df.to_string(index=False))

        if save_path:
            df.to_csv(save_path, index=False)
            print(f"\nReport saved to: {save_path}")

        return df

    def plot_detection_statistics(self, stats):
        """Create visualization of detection statistics"""
        if not stats:
            print("No statistics to plot")
            return

        # Prepare data
        classes = list(stats.keys())
        counts = [stats[cls]['total_detections'] for cls in classes]
        confidences = [stats[cls]['avg_confidence'] for cls in classes]

        # Create subplots
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

        # Detection counts
        ax1.bar(classes, counts, color='skyblue', alpha=0.7)
        ax1.set_title('Object Detection Counts')
        ax1.set_xlabel('Object Class')
        ax1.set_ylabel('Number of Detections')
        ax1.tick_params(axis='x', rotation=45)

        # Average confidence scores
        ax2.bar(classes, confidences, color='lightcoral', alpha=0.7)
        ax2.set_title('Average Confidence Scores')
        ax2.set_xlabel('Object Class')
        ax2.set_ylabel('Confidence Score')
        ax2.tick_params(axis='x', rotation=45)
        ax2.set_ylim(0, 1)

        plt.tight_layout()
        plt.show()

# ========================================
# DEMO AND TESTING FUNCTIONS
# ========================================

def demo_image_detection():
    """Demo: Object detection on sample images"""
    print("🎯 IMAGE DETECTION DEMO")
    print("=" * 40)

    # Initialize detector
    detector = ObjectDetector('n')  # Using nano model for speed

    # You can test with URLs or uploaded images
    print("Testing with sample image...")

    # For testing, you can use these sample images:
    sample_images = [
        "https://ultralytics.com/images/bus.jpg",
        "https://ultralytics.com/images/zidane.jpg"
    ]

    for i, img_url in enumerate(sample_images):
        print(f"\n--- Sample Image {i+1} ---")
        try:
            detections = detector.show_detections(img_url, confidence=0.4)
            print(f"Found {len(detections)} objects!")
        except Exception as e:
            print(f"Could not process image: {e}")

def demo_video_processing():
    """Demo: Video processing setup"""
    print("🎬 VIDEO PROCESSING DEMO")
    print("=" * 40)

    print("To process a video:")
    print("1. Upload your video file to Colab")
    print("2. Use the following code:")
    print("""
    # Initialize processor
    processor = VideoProcessor('n')

    # Process video
    frames, stats = processor.process_video_file(
        'your_video.mp4',
        'output_tracked.mp4',
        max_frames=100  # Limit for demo
    )

    # Analyze results
    analyzer = DetectionAnalyzer()
    detection_stats = analyzer.analyze_detections(frames)
    analyzer.create_detection_report(detection_stats)
    """)

def create_detection_interface():
    """Interactive interface for object detection"""
    print("🚀 OBJECT DETECTION & TRACKING SYSTEM")
    print("=" * 50)

    detector = ObjectDetector('n')

    while True:
        print("\nOptions:")
        print("1. Detect objects in image")
        print("2. Process video file")
        print("3. Show available object classes")
        print("4. Run image detection demo")
        print("5. Exit")

        choice = input("\nEnter your choice (1-5): ").strip()

        if choice == '1':
            img_path = input("Enter image path or URL: ").strip()
            confidence = float(input("Enter confidence threshold (0.1-1.0): ") or "0.5")

            print("Processing image...")
            detections = detector.show_detections(img_path, confidence)

        elif choice == '2':
            print("Video processing requires uploading a video file to Colab first.")
            video_path = input("Enter video file path: ").strip()

            if os.path.exists(video_path):
                processor = VideoProcessor('n')
                output_path = f"tracked_{os.path.basename(video_path)}"

                print("Processing video... This may take a while.")
                frames, stats = processor.process_video_file(
                    video_path,
                    output_path,
                    max_frames=100
                )

                print(f"✅ Video processed! Output saved as: {output_path}")
                print(f"📊 Statistics: {stats}")
            else:
                print("❌ Video file not found!")

        elif choice == '3':
            print("\n📋 Available Object Classes:")
            print("-" * 30)
            for i, (class_id, class_name) in enumerate(detector.class_names.items()):
                print(f"{class_id}: {class_name}")
                if i > 0 and (i + 1) % 10 == 0:  # Show 10 at a time
                    if input("\nPress Enter to continue or 'q' to stop: ") == 'q':
                        break

        elif choice == '4':
            demo_image_detection()

        elif choice == '5':
            print("Thank you for using the Object Detection System!")
            break

        else:
            print("Invalid choice. Please try again.")

# ========================================
# MAIN EXECUTION
# ========================================

if __name__ == "__main__":
    print("🎯 OBJECT DETECTION AND TRACKING SYSTEM")
    print("=" * 50)
    print("Initializing system...")

    # Run demo first
    demo_image_detection()

    # Start interactive interface
    print("\n" + "="*50)
    create_detection_interface()